In [2]:
import json
import pandas as pd
import re
from pathlib import Path

# ---------------------------------------------------------
# Cell 1: Setup & Load Data
# ---------------------------------------------------------

# Use the absolute path that worked for you in Task 1
# file_path = Path(r'F:\Programming\DataMiningProject\data\raw\result.json')
file_path = Path('../data/raw/result.json')

# Load JSON
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Normalize Structure
if isinstance(data, dict) and 'messages' in data:
    raw_messages = data['messages']
else:
    raw_messages = data

# Create DataFrame & Filter Messages
df = pd.DataFrame(raw_messages)
msgs = df[df['type'] == 'message'].copy()

# Clean Text Function (Flattening the Telegram structure)
def clean_telegram_text(text_obj):
    if isinstance(text_obj, str):
        return text_obj
    elif isinstance(text_obj, list):
        return ''.join([t if isinstance(t, str) else t.get('text', '') for t in text_obj])
    return ""

msgs['clean_text'] = msgs['text'].apply(clean_telegram_text)

print(f"Setup Complete. Loaded {len(msgs)} messages.")

Setup Complete. Loaded 4592 messages.


In [3]:
# Print 3 random long messages to see the exact structure
pd.set_option('display.max_colwidth', None)
print(msgs[msgs['clean_text'].str.len() > 200]['clean_text'].sample(3).values)

['🧑\u200d🏫 محسن صابر ماهانی\n🏫 #مهندسی_عمران\n📒 زمین شناسی مهندسی\n\nمنابع آموزش\n  ┘ یک کتاب مرجع معرفی می کند\n\nحضور و غیاب\n  ┘ حضور مهم است و تاثیر مستقیم دارد\n\nمنابع معرفی شده برای امتحان کافی است؟\n  ┘ بله\n\nوضعیت نمره دادن:\n  ┘ یادم نیست چجوری بود\n\nارزیابی دانشجو:\n  ┤ پیوستگی و یکپارچگی تدریس: 6\n  ┤ دانش عمومی: 9\n  ┤ توانایی انتقال مطالب و نکات مهم درس: 10\n  ┤ نحوه مدیریت کلاس(نظم و زمان): 10\n  ┤ پاسخگویی(حضوری و غیرحضوری): 1\n  ┤ آداب و رفتار اجتماعی با دانشجویان: 8\n\nراه ارتباطی:\n  ┘ چیزی ندارم\n\nترمی که دانشجو با این استاد کلاس داشته:\n  ┘ مهر ۱۴۰۰\n\nتوضیحات:\n  ┘ امتحان پایان ترم را شفاهی گرفتند\n~~~~~~~~~~~~~~~~~\nبرای ثبت معرفی استاد به ربات زیر پیام بدید\n@ostad_elmosiBot\n\nکانال معرفی اساتید دانشگاه علم و صنعت\n@ostad_elmosi @ostad_elmosi'
 '🧑\u200d🏫 امید نیکان\n🏫 #ریاضی\n📒 ریاضی عمومی ۲\n\nمنابع آموزش\n  ┘ فایل پاورپوینت یا پی دی اف می دهد\n\nحضور و غیاب\n  ┘ حضور و غیاب نمی کند\n\nمنابع معرفی شده برای امتحان کافی است؟\n  ┘ بله\n\nوضعیت نمره دادن:\n  ┘ 

In [19]:
import re
import numpy as np

def parse_review(text):
    """
    Final Version: Robust to Emojis, Spacing, and Multi-line Comments.
    """
    data = {
        'professor_name_raw': None,
        'department': None,
        'course_name': None,
        'grading_status_raw': None,
        'attendance_status_raw': None,
        'term': None,
        'comment_text': None,
        'parse_error': False
    }
    
    # Initialize ratings
    for i in range(1, 7):
        data[f'rating_{i}'] = np.nan

    try:
        if not isinstance(text, str) or len(text) < 50:
            data['parse_error'] = True
            return data

        # --- 1. Basic Info ---
        # Name (Robust to ZWJ and spacing)
        name_match = re.search(r'🧑\u200d?🏫\s*(?:استاد)?\s*(.+)', text)
        if name_match:
            data['professor_name_raw'] = name_match.group(1).strip()
        
        # Department (Requires Hashtag # to avoid matching Teacher emoji)
        dept_match = re.search(r'🏫\s*#(.+)', text)
        if dept_match:
            data['department'] = dept_match.group(1).strip().replace('_', ' ')

        # Course
        course_match = re.search(r'📒\s*(.+)', text)
        if course_match:
            data['course_name'] = course_match.group(1).strip()

        # --- 2. Standard Sections (Single Line) ---
        def get_section(header, text):
            pattern = re.escape(header) + r'.*?[\n\r]+\s*[┘└]\s*(.+)'
            match = re.search(pattern, text)
            return match.group(1).strip() if match else None

        data['grading_status_raw'] = get_section('وضعیت نمره دادن:', text)
        data['attendance_status_raw'] = get_section('حضور و غیاب', text)
        data['term'] = get_section('ترمی که دانشجو با این استاد کلاس داشته:', text)

        # --- 3. Comments (Multi-line Fix) ---
        # Capture everything from 'توضیحات' -> '┘' until the separator '~~~~'
        # re.DOTALL allows the dot (.) to match newlines
        comment_match = re.search(r'توضیحات:.*?[\n\r]+\s*[┘└]\s*(.*?)~{3,}', text, re.DOTALL)
        if comment_match:
            data['comment_text'] = comment_match.group(1).strip()
        else:
            # Fallback for older formats without '~~~~'
            data['comment_text'] = get_section('توضیحات:', text)

        # --- 4. Ratings ---
        ratings_map = {
            'پیوستگی و یکپارچگی تدریس': 'rating_1',
            'دانش عمومی': 'rating_2',
            'توانایی انتقال مطالب و نکات مهم درس': 'rating_3',
            'نحوه مدیریت کلاس(نظم و زمان)': 'rating_4',
            'پاسخگویی(حضوری و غیرحضوری)': 'rating_5',
            'آداب و رفتار اجتماعی با دانشجویان': 'rating_6'
        }

        rating_matches = re.findall(r'[┤\u2524]\s*(.+?):\s*(\d+)', text)
        for label, score in rating_matches:
            clean_label = label.strip()
            if clean_label in ratings_map:
                data[ratings_map[clean_label]] = int(score)

        if not data['professor_name_raw']:
            data['parse_error'] = True

    except Exception as e:
        data['parse_error'] = True
        print(f"Error parsing text: {e}")

    return data

print("Parser defined.")

Parser defined.


In [22]:
# ---------------------------------------------------------
# MASTER TEST SUITE
# 5 Edge Cases + 10 Random Real Samples
# ---------------------------------------------------------
import numpy as np

# --- PART A: EDGE CASE UNIT TESTS (5 Cases) ---
print("--- PART A: Checking 5 Edge Cases (Unit Tests) ---")

edge_cases = [
    {
        "desc": "✅ 1. Perfect Standard Review",
        "text": "🧑‍🏫 استاد علی منتظری\n🏫 #کامپیوتر\n📒 ساختمان داده\nارزیابی دانشجو:\n  ┤ دانش عمومی: 20\n  ┤ نحوه مدیریت کلاس(نظم و زمان): 18\n  ┤ پیوستگی و یکپارچگی تدریس: 19\n  ┤ توانایی انتقال مطالب و نکات مهم درس: 17\n  ┤ پاسخگویی(حضوری و غیرحضوری): 16\n  ┤ آداب و رفتار اجتماعی با دانشجویان: 20\nتوضیحات:\n  ┘ عالی بود"
    },
    {
        "desc": "⚠️ 2. Missing Optional Fields (No Course, No Comment)",
        "text": "🧑‍🏫 دکتر بی نام\n🏫 #ریاضی\nارزیابی دانشجو:\n  ┤ دانش عمومی: 10"
    },
    {
        "desc": "⚠️ 3. Weird Spacing & Formatting",
        "text": "🧑‍🏫   استاد    عجیب   \n🏫   #ریاضی   \n\nارزیابی دانشجو:\n┤دانش عمومی:20"
    },
    {
        "desc": "🧩 4. Messy Layout (Newlines between ratings)",
        "text": "🧑‍🏫 استاد نامرتب\n🏫 #برق\nارزیابی دانشجو:\n\n  ┤ دانش عمومی: 15\n\n\n  ┤ نحوه مدیریت کلاس(نظم و زمان): 14\n"
    },
    {
        "desc": "❌ 5. Non-Review Message (Should Error)",
        "text": "سلام بچه ها کلاس تشکیل میشه؟ این پیام اصلا نقد استاد نیست."
    }
]

for i, case in enumerate(edge_cases):
    res = parse_review(case['text'])
    
    # Determine Pass/Fail based on expectation
    is_error_expected = "Non-Review" in case['desc']
    status = "PASSED"
    
    if is_error_expected:
        if not res['parse_error']: status = "FAILED (Should have errored)"
    else:
        if res['parse_error']: status = "FAILED (Unexpected Error)"
        
    print(f"Test {case['desc']}: {status}")
    if not res['parse_error']:
        print(f"   -> Name: '{res['professor_name_raw']}' | Dept: '{res['department']}' | Ratings Found: {6 - list(res.values()).count(np.nan) - 8}") # rough count check


# --- PART B: REAL DATA SAMPLING (10 Samples) ---
print("\n" + "="*80)
print("--- PART B: Real Data Random Check (10 Samples) ---")
print("Displaying FULL data (no truncation)...")

# Pick 10 random messages that look like reviews (len > 150)
real_samples = msgs[msgs['clean_text'].str.len() > 150].sample(10, random_state=42)

for idx, row in real_samples.iterrows():
    raw_text = row['clean_text']
    parsed = parse_review(raw_text)
    
    print("\n" + "="*80)
    print(f"MESSAGE ID: {row['id']}")
    print("-" * 30 + " 1. RAW TEXT " + "-" * 30)
    print(raw_text.strip()) # Printing full text
    
    print("-" * 30 + " 2. PARSED RESULT " + "-" * 27)
    if parsed['parse_error']:
        print("RESULT: PARSE ERROR (Not a valid review)")
    else:
        print(f"👤 PROFESSOR:  {parsed['professor_name_raw']}")
        print(f"🏫 DEPT:       {parsed['department']}")
        print(f"📒 COURSE:     {parsed['course_name']}")
        print(f"🎓 TERM:       {parsed['term']}")
        
        print("\n--- RATINGS ---")
        print(f"1. Consistency (پیوستگی):   {parsed['rating_1']}")
        print(f"2. Knowledge (دانش عمومی):  {parsed['rating_2']}")
        print(f"3. Delivery (انتقال):       {parsed['rating_3']}")
        print(f"4. Management (مدیریت):     {parsed['rating_4']}")
        print(f"5. Responsiveness (پاسخ):   {parsed['rating_5']}")
        print(f"6. Ethics (آداب و رفتار):   {parsed['rating_6']}")
        
        print("\n--- COMMENTS ---")
        print(parsed['comment_text'])

print("\nEXTENDED VERIFICATION COMPLETE.")

--- PART A: Checking 5 Edge Cases (Unit Tests) ---
Test ✅ 1. Perfect Standard Review: PASSED
   -> Name: 'علی منتظری' | Dept: 'کامپیوتر' | Ratings Found: -2
Test ⚠️ 2. Missing Optional Fields (No Course, No Comment): PASSED
   -> Name: 'دکتر بی نام' | Dept: 'ریاضی' | Ratings Found: -7
Test ⚠️ 3. Weird Spacing & Formatting: PASSED
   -> Name: 'عجیب' | Dept: 'ریاضی' | Ratings Found: -7
Test 🧩 4. Messy Layout (Newlines between ratings): PASSED
   -> Name: 'نامرتب' | Dept: 'برق' | Ratings Found: -6
Test ❌ 5. Non-Review Message (Should Error): PASSED

--- PART B: Real Data Random Check (10 Samples) ---
Displaying FULL data (no truncation)...

MESSAGE ID: 1376
------------------------------ 1. RAW TEXT ------------------------------
🧑‍🏫 استاد سمانه مشهدی
🏫 #ریاضی
📒 مبانی ریاضی مقدماتی

منابع آموزش
  ┘ یک کتاب مرجع معرفی می کند

حضور و غیاب
  ┘ یادم نمیاد

منابع معرفی شده برای امتحان کافی است؟
  ┘ خیر

وضعیت نمره دادن:
  ┘ نمره خوبی نمیشه ازشون گرفت

ارزیابی دانشجو:
  ┤ پیوستگی و یکپارچگی تدر

In [23]:
# ---------------------------------------------------------
# STRESS TEST SUITE: Rejections & Extremes (Clean Logs)
# ---------------------------------------------------------
import matplotlib.pyplot as plt

# Apply parser to ALL data first
all_parsed = msgs['clean_text'].apply(parse_review).tolist()
df_all = pd.DataFrame(all_parsed)
df_full = pd.concat([msgs[['id', 'clean_text']].reset_index(drop=True), df_all], axis=1)

# --- TEST 1: The "False Negative" Check ---
failed_rows = df_full[df_full['parse_error'] == True]
print(f"Rejected Messages: {len(failed_rows)} out of {len(df_full)}")

if len(failed_rows) > 0:
    print("\nINSPECTING 5 REJECTED SAMPLES (Should NOT look like reviews):")
    # Ignore very short messages (<50 chars)
    potential_misses = failed_rows[failed_rows['clean_text'].str.len() > 50]
    
    if len(potential_misses) > 0:
        for idx, row in potential_misses.sample(min(5, len(potential_misses)), random_state=42).iterrows():
            print(f"   [ID {row['id']}] {row['clean_text'][:80]}...")
    else:
        print("   All failed messages were very short (likely chat spam).")
else:
    print("   No rejections found.")


# --- TEST 2: The "Extremes" Check ---
# Added .copy() to prevent SettingWithCopyWarning
valid_rows = df_full[df_full['parse_error'] == False].copy()

print("\n" + "="*50)
print("EXTREME VALUE CHECKS (Valid Reviews Only)")

# A. Shortest & Longest Professor Names
valid_rows['name_len'] = valid_rows['professor_name_raw'].str.len()
shortest_name = valid_rows.nsmallest(1, 'name_len')
longest_name = valid_rows.nlargest(1, 'name_len')

print(f"\nShortest Name Found ({shortest_name.iloc[0]['name_len']} chars):")
print(f"   '{shortest_name.iloc[0]['professor_name_raw']}'") 

print(f"Longest Name Found ({longest_name.iloc[0]['name_len']} chars):")
print(f"   '{longest_name.iloc[0]['professor_name_raw']}'")

# B. Empty Departments?
missing_dept = valid_rows[valid_rows['department'].isna()]
print(f"\nReviews with NO Department: {len(missing_dept)} ({len(missing_dept)/len(valid_rows):.1%}%)")
if len(missing_dept) > 0:
    print(f"   Sample ID with missing dept: {missing_dept.iloc[0]['id']}")

# C. Out of Bounds Scores
# Check if any score column has values outside 0-20
rating_cols = [c for c in valid_rows.columns if 'rating_' in c]
print("\nScore Range Check (Should be 0-20):")
for col in rating_cols:
    min_val = valid_rows[col].min()
    max_val = valid_rows[col].max()
    status = "OK" if (min_val >= 0 and max_val <= 20) else "ALERT"
    print(f"   {col}: Min={min_val} | Max={max_val} [{status}]")

print("\nSTRESS TEST COMPLETE.")

Rejected Messages: 44 out of 4592

INSPECTING 5 REJECTED SAMPLES (Should NOT look like reviews):
   [ID 26] سلام دوستان این ترم هم مثل تمامی ترم ها نیاز به #معرفی_استاد داریم

لطفا استاد ه...
   [ID 1176] #رنج_نمرات
👨‍🏫استاد حسینی مطلق
🏫 صنایع
📒 تحقیق در عملیات ۱
ترم بهمن ۱۴۰۰

کانال ...
   [ID 12] #خبر_مهم

سلام به همه‌ی علموصی ها😍👊🏾

✅این ربات برای فرآیند #انتخاب_واحد آماده ش...
   [ID 1818] لطفاً رنج نمرات کلاس خود را برای قرارگیری در کانال معرفی اساتید، تنها به این آید...
   [ID 66] خودتون به معرفی ها اعتبار بدید!
اگر از استادی خوب گفتن و همه مخالفن چیو نشون مید...

EXTREME VALUE CHECKS (Valid Reviews Only)

Shortest Name Found (3 chars):
   'نصر'
Longest Name Found (32 chars):
   'سید علیرضا نظام الحسینی عز آبادی'

Reviews with NO Department: 27 (0.6%%)
   Sample ID with missing dept: 816

Score Range Check (Should be 0-20):
   rating_1: Min=1.0 | Max=10.0 [OK]
   rating_2: Min=1.0 | Max=10.0 [OK]
   rating_3: Min=1.0 | Max=10.0 [OK]
   rating_4: Min=1.0 | Max=10.0 [OK]
   rating_

In [25]:
# ---------------------------------------------------------
# FINAL EXECUTION: Apply Parser to All Data & Save
# ---------------------------------------------------------
from pathlib import Path

# 1. Apply parser to all messages
parsed_list = msgs['clean_text'].apply(parse_review).tolist()
df_parsed = pd.DataFrame(parsed_list)

# 2. Merge with original metadata (ID, Date)
df_combined = pd.concat([msgs[['id', 'date']].reset_index(drop=True), df_parsed], axis=1)

# 3. Filter: Keep only valid reviews (drop parse errors)
valid_reviews = df_combined[df_combined['parse_error'] == False].drop(columns=['parse_error'])

# 4. Save to CSV
output_path = Path(r'F:\Programming\DataMiningProject\data\processed\parsed_data.csv')
output_path.parent.mkdir(parents=True, exist_ok=True)

# Use utf-8-sig for correct Persian display in Excel/Windows
valid_reviews.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"[SUCCESS] Saved {len(valid_reviews)} valid reviews.")
print(f"Location: {output_path}")

[SUCCESS] Saved 4548 valid reviews.
Location: F:\Programming\DataMiningProject\data\processed\parsed_data.csv
